In [ ]:
# Sipmle Stacks

#Simple Stack Problem
from collections import deque
# https://leetcode.com/problems/remove-all-adjacent-d
# uplicates-in-string-ii/description/
def removeDuplicates(s: str, k: int) -> str:
    stack = deque()
    for char in s:
        if stack and stack[-1][0] ==char:
            stack[-1][1] += 1
            if stack[-1][1] == k:
                stack.pop()
        else:
            stack.append([char, 1])
    return "".join([x[0]*x[1] for x in stack])

In [1]:
def balance_parentheses(s):
    # we shall use a stack based approach 
    # I will delete the unmatched brackets and return the result
    # will do this in two passes
    # first pass: delete the unmatched right brackets
    # and also collect all the unmatched left brackets
    # second pass we traverse the stack: delete the unmatched left brackets
    stack = []
    char_array = list(s)# Convert to list so we can mark deletions easily
    # Yess Its an array in python 

    for i, char in enumerate(char_array):
        if char == '(':
            stack.append(i) # store the index of the left bracket
        elif char == ')':
            if stack: # non empty
                stack.pop() 
            else:
                char_array[i] = '' # we are deleting by replacing with empty string
    while stack:
        index = stack.pop()
        char_array[index] = '' # we are deleting by replacing with empty string
    return "".join(char_array)

print(balance_parentheses("((())"))
 

(())


In [ ]:


# Map data strucutre with nestedd transactions
# Trnasactions means -- all or nothing -- i.e. all operations must either be completed or none
# Question 
# Abort will it undo all the open transactions -- even the comitted ones
# All open transactions will be undone ??
# If we abort one transactoin what will happen to transactoins inside it? 

class MapWithTransactions:
    def __init__(self):
        self._base = {} # Base dictionary to store committed data outside of transactions
        self._tx_stack = [] # Stack of transaction dicts still open (not yet committed)

     # O(1) 
    def begin_tx(self): # Start a new (nested) transaction by pushing an empty dict to the stack.
        self._tx_stack.append({}) 
    
    # O(k)  -- where k is the numebr of entries in the top most txn
    def commit_tx(self): 
        # Commit the topmost transaction: 
        #     (if nested) merge its changes into the pareent transaction 
        #     (else base level) into the base dictionary (if top-level). 
        top_changes = self._tx_stack.pop()
        if self._tx_stack: # If there is a lower transaction, merge changes into it 
            # Thus committing merges this transaction into the parent transaction
            for key, value in top_changes.items(): # -1 will be the last transaction i.e. parent
                self._tx_stack[-1][key] = value
        else: # w
            for key, value in top_changes.items():
                self._base[key] = value
    
    # O(1) 
    def abort_tx(self): #  Abort the topmost transaction: discard all its changes.
        self._tx_stack.pop()
    
     # O(1) 
    def set(self, key, value): # Set a key to a given value, either in the topmost transaction or in base.
        if self._tx_stack:
            # If there is an active transaction, record change there
            self._tx_stack[-1][key] = value
        else:
            # Otherwise, just set in the base
            self._base[key] = value

     # O(T) -- T is the number of current transactions
    def get(self, key): #Return the most recent value of 'key' from topmost transaction down.
        for tx_dict in reversed(self._tx_stack): # top most transaction first
            if key in tx_dict:  
                res = tx_dict[key]  # Could be None if "deleted"
                print(key, res)
                return res
        res =  self._base.get(key, None) # If not found in any transaction, look in base
        print(key, res)
        return res

# Spacxe Complixity -- O(M)  -- M totoal number of steps across all non-comittedd transactions + Size of base Dictionary(committed data)
# by step I mean the set operation

# Compleity -- n size map
# k transactions 
# space coplyx - kn
# time complexity 
# ------------------------
# Example usage:
# ------------------------
 
m = MapWithTransactions()

# Set a key in the base store
m.set("foo", 1)

# Begin a new transaction
m.begin_tx()
m.set("foo", 42)
m.set("bar", 123)
m.get("foo") # 42 (uncommitted)
m.get("bar") # 123 (uncommitted)

# Commit the transaction
m.commit_tx()
m.get("foo")  # 42 (committed)
m.get("bar")  # 123 (committed)

# Start another transaction
m.begin_tx()
m.set("foo", 13)
m.get("foo")  # 13 (uncommitted)

# Abort the transaction
m.abort_tx()
print(m.get("foo"))  # 42 (previous committed value)


